In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [ ]:
%%writefile streamlitapp.py
import streamlit as st
import numpy as np
from PIL import Image
import tensorflow as tf
# Load your trained model
model = tf.keras.models.load_model("/content/handwritten_digit_model.h5")
def main():
    st.title("MNIST Handwritten Digits Streamlit Interface") #title of the Streamlit webpage

    # Allow users to upload their own image
    uploaded_file = st.file_uploader("Upload Your Own Image", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None: #uploaded file exists
        # Display the uploaded image
        uploaded_image = Image.open(uploaded_file)
        st.image(uploaded_image, caption="Uploaded Image", use_column_width=True)

        #preprocess and pass the input to your model
        #similar to Gradio
        raw_image = np.array(uploaded_image)
        reshaped_image = np.reshape(raw_image, (-1, 28, 28, 1))
        prediction_raw = model.predict(reshaped_image)
        predicted_label = np.argmax(prediction_raw)

        # Display the predicted label
        st.write("Prediction:", predicted_label)

if __name__ == "__main__":
    main()

Writing streamlitapp.py


In [ ]:
!npm install -g localtunnel
!streamlit run /content/streamlitapp.py &>/content/logs.txt &
!lt --port 8501 & curl ipv4.icanhazip.com

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.801s
35.230.77.186
your url is: https://warm-laws-open.loca.lt


In [ ]:
!pip install flask

In [ ]:
%%writefile app.py
from flask import Flask, render_template, request, jsonify
import numpy as np
from PIL import Image
import base64
import os
from io import BytesIO
from tensorflow.keras.models import load_model
app = Flask(__name__)

model = load_model('handwritten_digit_model.h5')  # Replace with your model path

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    # Get the image file from the request
    image_file = request.files['file']

    # Process the image
    img = Image.open(image_file).convert('L')  # Convert to grayscale
    img = img.resize((28, 28))  # Resize to MNIST input size
    img_array = np.array(img).reshape(1, 28, 28, 1)  # Normalize

    # Make prediction
    prediction = model.predict(img_array)
    predicted_digit = np.argmax(prediction)

    # Convert the PIL Image to base64-encoded bytes
    img_bytes = BytesIO()
    img.save(img_bytes, format='PNG')
    img_base64 = base64.b64encode(img_bytes.getvalue()).decode('utf-8')

    # Create a response with the base64-encoded image and prediction
    response = {
        'prediction': int(predicted_digit),
        'image': {
            'format': 'png',
            'data': img_base64
        }
    }

    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True)

Writing app.py


In [ ]:
!npm install -g localtunnel
!flask run &>/content/logs.txt &
!lt --port 5000 & curl ipv4.icanhazip.com

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 0.987s


   ╭────────────────────────────────────────────────────────────────╮
   │                                                                │
   │      New major version of npm available! 6.14.8 → 10.2.5       │
   │   Changelog: https://github.com/npm/cli/releases/tag/v10.2.5   │
   │               Run npm install -g npm to update!                │
   │                                                                │
   ╰────────────────────────────────────────────────────────────────╯

35.230.77.186
your url is: https://mean-pants-kiss.loca.lt
